In [1]:
# Célula 6: Configuração do AWS Rekognition
import boto3
from botocore.exceptions import ClientError
import json

# Configurar cliente Rekognition
try:
    rekognition_client = boto3.client('rekognition', region_name='us-east-1')
    print("✅ Cliente AWS Rekognition configurado com sucesso!")
    
    # Teste de conectividade
    response = rekognition_client.list_collections()
    print(f"🔗 Conexão com Rekognition estabelecida. Collections disponíveis: {len(response.get('CollectionIds', []))}")
    
except Exception as e:
    print(f"❌ Erro ao configurar Rekognition: {e}")

def detect_faces_in_image(bucket_name, image_key):
    """
    Detecta faces em imagem armazenada no S3
    
    Args:
        bucket_name (str): Nome do bucket S3
        image_key (str): Chave/nome do arquivo no S3
    
    Returns:
        list: Lista de detalhes das faces detectadas
    """
    try:
        response = rekognition_client.detect_faces(
            Image={
                'S3Object': {
                    'Bucket': bucket_name,
                    'Name': image_key
                }
            },
            Attributes=['ALL']  # Retorna todos os atributos faciais
        )
        
        faces = response['FaceDetails']
        print(f"🎯 {len(faces)} face(s) detectada(s) na imagem '{image_key}'")
        
        return faces
        
    except ClientError as e:
        print(f"❌ Erro do AWS Rekognition: {e}")
        return []
    except Exception as e:
        print(f"❌ Erro geral na detecção: {e}")
        return []

print("📦 Função detect_faces_in_image() definida com sucesso!")

✅ Cliente AWS Rekognition configurado com sucesso!
🔗 Conexão com Rekognition estabelecida. Collections disponíveis: 0
📦 Função detect_faces_in_image() definida com sucesso!


In [1]:
# Célula 6.1: Instalação das dependências necessárias
import subprocess
import sys

def install_packages():
    """Instala pacotes necessários para processamento de imagem"""
    packages = [
        'opencv-python',
        'Pillow',
        'matplotlib',
        'numpy'
    ]
    
    for package in packages:
        print(f"📦 Instalando {package}...")
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
            print(f"✅ {package} instalado com sucesso!")
        except subprocess.CalledProcessError as e:
            print(f"❌ Erro ao instalar {package}: {e}")

# Executar instalação
install_packages()

print("\n🎉 Instalação concluída! Reinicie o kernel do Jupyter e execute novamente.")

📦 Instalando opencv-python...


✅ opencv-python instalado com sucesso!
📦 Instalando Pillow...


✅ Pillow instalado com sucesso!
📦 Instalando matplotlib...


✅ matplotlib instalado com sucesso!
📦 Instalando numpy...
✅ numpy instalado com sucesso!

🎉 Instalação concluída! Reinicie o kernel do Jupyter e execute novamente.


In [1]:
# Célula 6.1: Pré-processamento da Imagem
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def preprocess_document_image(image_path):
    """
    Pré-processa imagem de documento para melhor OCR e detecção
    """
    # Carregar imagem
    img = cv2.imread(image_path)
    
    # 1. Converter para RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # 2. Aumentar contraste para melhor OCR
    lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    enhanced = cv2.merge([l, a, b])
    enhanced = cv2.cvtColor(enhanced, cv2.COLOR_LAB2RGB)
    
    # 3. Redimensionar se muito grande (AWS tem limites)
    height, width = enhanced.shape[:2]
    if width > 4096 or height > 4096:
        scale = min(4096/width, 4096/height)
        new_width = int(width * scale)
        new_height = int(height * scale)
        enhanced = cv2.resize(enhanced, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)
    
    return enhanced

def save_processed_image(processed_img, output_path):
    """Salva imagem processada"""
    pil_img = Image.fromarray(processed_img)
    pil_img.save(output_path, 'JPEG', quality=95)
    return output_path

print("📦 Funções de pré-processamento definidas!")

📦 Funções de pré-processamento definidas!


In [5]:
# Célula 7 CORRIGIDA: Sistema completo com todas as importações
import boto3
from botocore.exceptions import ClientError
import os
from pathlib import Path
import mimetypes
import json

# Reconfigurar clientes AWS (caso tenham sido perdidos)
try:
    # Cliente S3
    s3_client = boto3.client('s3', region_name='us-east-1')
    print("✅ Cliente S3 reconfigurado!")
    
    # Verificar se bucket existe
    bucket_name = 'staging-face-text'
    s3_client.head_bucket(Bucket=bucket_name)
    print(f"✅ Bucket '{bucket_name}' acessível!")
    
    # Cliente Rekognition (reconfirmar)
    rekognition_client = boto3.client('rekognition', region_name='us-east-1')
    print("✅ Cliente Rekognition reconfigurado!")
    
except ClientError as e:
    print(f"❌ Erro de configuração AWS: {e}")
except Exception as e:
    print(f"❌ Erro geral: {e}")

def upload_image_to_s3(file_path, bucket_name, object_key=None):
    """
    Upload de imagem para S3 com validação completa
    """
    try:
        # Validar se arquivo existe
        if not os.path.exists(file_path):
            print(f"❌ Arquivo não encontrado: {file_path}")
            return False, None
        
        # Validar tipo de arquivo
        mime_type, _ = mimetypes.guess_type(file_path)
        if not mime_type or not mime_type.startswith('image/'):
            print(f"❌ Arquivo não é uma imagem válida: {mime_type}")
            return False, None
        
        # Definir object_key se não fornecido
        if object_key is None:
            object_key = Path(file_path).name
        
        # Upload para S3
        s3_client.upload_file(
            file_path, 
            bucket_name, 
            object_key,
            ExtraArgs={
                'ContentType': mime_type,
                'Metadata': {
                    'uploaded-by': 'face-text-detector',
                    'original-name': Path(file_path).name
                }
            }
        )
        
        print(f"✅ Upload concluído!")
        print(f"   📁 Arquivo local: {file_path}")
        print(f"   ☁️  S3 Object: s3://{bucket_name}/{object_key}")
        
        return True, object_key
        
    except ClientError as e:
        print(f"❌ Erro AWS S3: {e}")
        return False, None
    except Exception as e:
        print(f"❌ Erro no upload: {e}")
        return False, None

def list_bucket_contents(bucket_name):
    """Lista conteúdo do bucket S3"""
    try:
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        if 'Contents' in response:
            print(f"📂 Conteúdo do bucket '{bucket_name}':")
            for obj in response['Contents']:
                print(f"   📄 {obj['Key']} ({obj['Size']} bytes)")
        else:
            print(f"📂 Bucket '{bucket_name}' está vazio")
    except Exception as e:
        print(f"❌ Erro ao listar bucket: {e}")

# Importar funções de pré-processamento (caso necessário)
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def preprocess_document_image(image_path):
    """Pré-processa imagem de documento para melhor OCR e detecção"""
    try:
        # Carregar imagem
        img = cv2.imread(image_path)
        if img is None:
            print(f"❌ Não foi possível carregar a imagem: {image_path}")
            return None
        
        # Converter para RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Aumentar contraste para melhor OCR
        lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        l = clahe.apply(l)
        enhanced = cv2.merge([l, a, b])
        enhanced = cv2.cvtColor(enhanced, cv2.COLOR_LAB2RGB)
        
        # Redimensionar se muito grande (AWS tem limites)
        height, width = enhanced.shape[:2]
        if width > 4096 or height > 4096:
            scale = min(4096/width, 4096/height)
            new_width = int(width * scale)
            new_height = int(height * scale)
            enhanced = cv2.resize(enhanced, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)
            print(f"📏 Imagem redimensionada: {width}x{height} → {new_width}x{new_height}")
        
        return enhanced
        
    except Exception as e:
        print(f"❌ Erro no pré-processamento: {e}")
        return None

def save_processed_image(processed_img, output_path):
    """Salva imagem processada"""
    try:
        pil_img = Image.fromarray(processed_img)
        pil_img.save(output_path, 'JPEG', quality=95)
        print(f"💾 Imagem salva: {output_path}")
        return output_path
    except Exception as e:
        print(f"❌ Erro ao salvar imagem: {e}")
        return None

def process_your_infographic():
    """Processa especificamente sua imagem de infográfico"""
    
    # Localizar imagens no diretório atual
    current_dir = os.getcwd()
    print(f"📁 Procurando imagens em: {current_dir}")
    
    # Listar arquivos de imagem
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']
    found_images = []
    
    for file in os.listdir(current_dir):
        if any(file.lower().endswith(ext) for ext in image_extensions):
            found_images.append(file)
    
    if found_images:
        print(f"📸 Imagens encontradas:")
        for i, img in enumerate(found_images, 1):
            print(f"   {i}. {img}")
        
        # Usar a primeira imagem encontrada
        target_image = found_images[0]
        print(f"\n🎯 Processando: {target_image}")
        
        # Pré-processar a imagem
        print("🔧 Aplicando pré-processamento...")
        processed_img = preprocess_document_image(target_image)
        
        if processed_img is not None:
            # Salvar versão processada
            processed_filename = f"processed_{target_image}"
            processed_path = save_processed_image(processed_img, processed_filename)
            
            if processed_path:
                # Fazer upload da versão processada
                print("☁️ Fazendo upload para S3...")
                success, s3_key = upload_image_to_s3(processed_path, 'staging-face-text')
                
                if success:
                    return processed_path, s3_key
                else:
                    return None, None
            else:
                return None, None
        else:
            print("❌ Falha no pré-processamento")
            return None, None
    else:
        print("❌ Nenhuma imagem encontrada no diretório atual")
        print("💡 Coloque sua imagem na pasta do projeto e execute novamente")
        return None, None

print("📦 Sistema completo configurado com sucesso!")
print("🚀 Pronto para processar sua imagem!")

✅ Cliente S3 reconfigurado!
✅ Bucket 'staging-face-text' acessível!
✅ Cliente Rekognition reconfigurado!
📦 Sistema completo configurado com sucesso!
🚀 Pronto para processar sua imagem!


In [6]:
# Teste completo
print("🧪 Iniciando processamento da sua imagem...")
processed_path, s3_key = process_your_infographic()

if s3_key:
    print(f"\n🎉 SUCESSO!")
    print(f"📁 Arquivo local processado: {processed_path}")
    print(f"☁️  Arquivo no S3: {s3_key}")
    print("🚀 Pronto para detecção facial e OCR!")
    
    # Listar conteúdo do bucket
    print("\n📂 Verificando bucket:")
    list_bucket_contents('staging-face-text')
else:
    print("❌ Falha no processamento. Verifique se há imagens na pasta.")

🧪 Iniciando processamento da sua imagem...
📁 Procurando imagens em: /app
📸 Imagens encontradas:
   1. Pasted image 20250608090919.png
   2. processed_Pasted image 20250608090919.png

🎯 Processando: Pasted image 20250608090919.png
🔧 Aplicando pré-processamento...
💾 Imagem salva: processed_Pasted image 20250608090919.png
☁️ Fazendo upload para S3...
✅ Upload concluído!
   📁 Arquivo local: processed_Pasted image 20250608090919.png
   ☁️  S3 Object: s3://staging-face-text/processed_Pasted image 20250608090919.png

🎉 SUCESSO!
📁 Arquivo local processado: processed_Pasted image 20250608090919.png
☁️  Arquivo no S3: processed_Pasted image 20250608090919.png
🚀 Pronto para detecção facial e OCR!

📂 Verificando bucket:
📂 Conteúdo do bucket 'staging-face-text':
   📄 processed_Pasted image 20250608090919.png (136758 bytes)


In [9]:
# Célula 8 CORRIGIDA: Sistema completo de OCR com AWS Textract
import boto3
from botocore.exceptions import ClientError
import json
from datetime import datetime

# Configurar cliente Textract
try:
    textract_client = boto3.client('textract', region_name='us-east-1')
    print("✅ Cliente AWS Textract configurado com sucesso!")
    
except Exception as e:
    print(f"❌ Erro ao configurar Textract: {e}")

def extract_text_from_s3_image(bucket_name, image_key):
    """
    Extrai todo o texto de uma imagem no S3 usando AWS Textract
    """
    try:
        print(f"🔍 Iniciando OCR da imagem: {image_key}")
        
        # Executar OCR com Textract
        response = textract_client.detect_document_text(
            Document={
                'S3Object': {
                    'Bucket': bucket_name,
                    'Name': image_key
                }
            }
        )
        
        # Processar resultados
        extracted_text = ""
        text_blocks = []
        words = []
        lines = []
        
        for block in response['Blocks']:
            if block['BlockType'] == 'LINE':
                line_text = block['Text']
                extracted_text += line_text + "\n"
                lines.append({
                    'text': line_text,
                    'confidence': round(block['Confidence'], 2),
                    'bounding_box': block['Geometry']['BoundingBox']
                })
            
            elif block['BlockType'] == 'WORD':
                words.append({
                    'text': block['Text'],
                    'confidence': round(block['Confidence'], 2),
                    'bounding_box': block['Geometry']['BoundingBox']
                })
        
        # Compilar resultados
        result = {
            'image_key': image_key,
            'bucket_name': bucket_name,
            'full_text': extracted_text.strip(),
            'total_lines': len(lines),
            'total_words': len(words),
            'lines': lines,
            'words': words,
            'extraction_timestamp': datetime.now().isoformat(),
            'aws_response_metadata': response.get('ResponseMetadata', {})
        }
        
        # Estatísticas de confiança
        if words:
            confidences = [word['confidence'] for word in words]
            result['confidence_stats'] = {
                'average': round(sum(confidences) / len(confidences), 2),
                'min': min(confidences),
                'max': max(confidences)
            }
        
        print(f"📊 OCR CONCLUÍDO!")
        print(f"   📝 Linhas de texto: {len(lines)}")
        print(f"   🔤 Palavras detectadas: {len(words)}")
        print(f"   🎯 Confiança média: {result.get('confidence_stats', {}).get('average', 'N/A')}%")
        
        return result
        
    except ClientError as e:
        print(f"❌ Erro do AWS Textract: {e}")
        return None
    except Exception as e:
        print(f"❌ Erro na extração de texto: {e}")
        return None

def display_ocr_results(ocr_result):
    """Exibe resultados do OCR de forma organizada"""
    if not ocr_result:
        print("❌ Nenhum resultado de OCR para exibir")
        return
    
    print("\n" + "="*60)
    print("📄 RESULTADOS DO OCR (TEXTRACT)")
    print("="*60)
    
    print(f"📸 Imagem: {ocr_result['image_key']}")
    print(f"🕐 Processado em: {ocr_result['extraction_timestamp']}")
    print(f"📊 Estatísticas:")
    print(f"   • Linhas: {ocr_result['total_lines']}")
    print(f"   • Palavras: {ocr_result['total_words']}")
    
    if 'confidence_stats' in ocr_result:
        stats = ocr_result['confidence_stats']
        print(f"   • Confiança: {stats['average']}% (min: {stats['min']}%, max: {stats['max']}%)")
    
    print(f"\n📝 TEXTO EXTRAÍDO:")
    print("-" * 40)
    print(ocr_result['full_text'])
    print("-" * 40)
    
    # Mostrar linhas com baixa confiança (CORRIGIDO)
    low_confidence_lines = [line for line in ocr_result['lines'] if line['confidence'] < 80]
    if low_confidence_lines:
        print(f"\n⚠️  LINHAS COM BAIXA CONFIANÇA (<80%):")
        for line in low_confidence_lines:
            print(f"   • '{line['text']}' (confiança: {line['confidence']}%)")

def save_ocr_results_to_s3(ocr_result, bucket_name):
    """Salva resultados do OCR no S3"""
    try:
        # Nome do arquivo de resultados
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_key = f"ocr_results_{timestamp}.json"
        
        # Salvar no S3
        s3_client.put_object(
            Bucket=bucket_name,
            Key=results_key,
            Body=json.dumps(ocr_result, indent=2, ensure_ascii=False),
            ContentType='application/json',
            Metadata={
                'content-type': 'ocr-results',
                'source-image': ocr_result['image_key']
            }
        )
        
        print(f"💾 Resultados salvos: s3://{bucket_name}/{results_key}")
        return results_key
        
    except Exception as e:
        print(f"❌ Erro ao salvar resultados: {e}")
        return None

print("📦 Sistema de OCR configurado com sucesso!")

✅ Cliente AWS Textract configurado com sucesso!
📦 Sistema de OCR configurado com sucesso!


In [10]:
# Executar OCR na sua imagem processada
bucket_name = 'staging-face-text'
image_key = 'processed_Pasted image 20250608090919.png'

print("🔍 Executando OCR na sua imagem...")
ocr_results = extract_text_from_s3_image(bucket_name, image_key)

if ocr_results:
    # Exibir resultados
    display_ocr_results(ocr_results)
    
    # Salvar resultados no S3
    results_key = save_ocr_results_to_s3(ocr_results, bucket_name)
    
    print(f"\n🎉 OCR CONCLUÍDO COM SUCESSO!")
    print(f"📄 Todo o texto do seu infográfico foi extraído!")
    
else:
    print("❌ Falha na extração de texto")

🔍 Executando OCR na sua imagem...
🔍 Iniciando OCR da imagem: processed_Pasted image 20250608090919.png
📊 OCR CONCLUÍDO!
   📝 Linhas de texto: 36
   🔤 Palavras detectadas: 143
   🎯 Confiança média: 84.13%

📄 RESULTADOS DO OCR (TEXTRACT)
📸 Imagem: processed_Pasted image 20250608090919.png
🕐 Processado em: 2025-06-10T16:09:13.888821
📊 Estatísticas:
   • Linhas: 36
   • Palavras: 143
   • Confiança: 84.13% (min: 7.03%, max: 99.94%)

📝 TEXTO EXTRAÍDO:
----------------------------------------
FIAP MBA+
FACE & TEXT EXTRACTION
o setor de fraudes apontou que existem clientes que se queixaram de não contratar serviços
específicos, como o crédito pessoal. Entretanto após o indicador de Detecção de vivacidade
(liveness), desenvolvido na disciplina de Computer Vision, ter apresentado um percentual de
vivacidade menor que 90% apontou a necessidade de uma nova validação do self da pessoa com o
documento.
REPUBLICA FEDERATIVA 00 BRASIL
BR
SECRETARIA NACIONA TRANSITO
CARTERA NACIONAL DE HABULTACIO DRIV

In [12]:
# Célula 9 CORRIGIDA: Sistema completo de detecção facial
import boto3
from botocore.exceptions import ClientError

# Reconfigurar cliente Rekognition (caso necessário)
try:
    rekognition_client = boto3.client('rekognition', region_name='us-east-1')
    print("✅ Cliente Rekognition reconfigurado!")
except Exception as e:
    print(f"❌ Erro ao configurar Rekognition: {e}")

def detect_faces_in_image(bucket_name, image_key):
    """
    Detecta faces em imagem armazenada no S3
    
    Args:
        bucket_name (str): Nome do bucket S3
        image_key (str): Chave/nome do arquivo no S3
    
    Returns:
        list: Lista de detalhes das faces detectadas
    """
    try:
        print(f"🔍 Analisando faces na imagem: {image_key}")
        
        response = rekognition_client.detect_faces(
            Image={
                'S3Object': {
                    'Bucket': bucket_name,
                    'Name': image_key
                }
            },
            Attributes=['ALL']  # Retorna todos os atributos faciais
        )
        
        faces = response['FaceDetails']
        print(f"🎯 {len(faces)} face(s) detectada(s) na imagem")
        
        return faces
        
    except ClientError as e:
        print(f"❌ Erro do AWS Rekognition: {e}")
        return []
    except Exception as e:
        print(f"❌ Erro geral na detecção: {e}")
        return []

def analyze_face_details(face, face_number):
    """Analisa detalhes de uma face específica"""
    print(f"\n👤 FACE {face_number}:")
    print(f"   🎯 Confiança geral: {face['Confidence']:.2f}%")
    
    # Posição da face
    bbox = face['BoundingBox']
    print(f"   📍 Posição: Left={bbox['Left']:.3f}, Top={bbox['Top']:.3f}")
    print(f"   📏 Dimensões: Width={bbox['Width']:.3f}, Height={bbox['Height']:.3f}")
    
    # Idade estimada
    if 'AgeRange' in face:
        age = face['AgeRange']
        print(f"   🎂 Idade estimada: {age['Low']}-{age['High']} anos")
    
    # Gênero
    if 'Gender' in face:
        gender = face['Gender']
        print(f"   👫 Gênero: {gender['Value']} (confiança: {gender['Confidence']:.1f}%)")
    
    # Emoções
    if 'Emotions' in face:
        emotions = sorted(face['Emotions'], key=lambda x: x['Confidence'], reverse=True)
        print(f"   😊 Emoções detectadas:")
        for emotion in emotions[:3]:  # Top 3 emoções
            print(f"      • {emotion['Type']}: {emotion['Confidence']:.1f}%")
    
    # Qualidade da imagem
    if 'Quality' in face:
        quality = face['Quality']
        print(f"   📸 Qualidade da imagem:")
        print(f"      • Brilho: {quality['Brightness']:.1f}")
        print(f"      • Nitidez: {quality['Sharpness']:.1f}")
    
    # Atributos faciais
    facial_attributes = ['Smile', 'Eyeglasses', 'Sunglasses', 'Beard', 'Mustache']
    detected_attributes = []
    
    for attr in facial_attributes:
        if attr in face and face[attr]['Value']:
            confidence = face[attr]['Confidence']
            detected_attributes.append(f"{attr} ({confidence:.1f}%)")
    
    if detected_attributes:
        print(f"   👁️  Atributos detectados: {', '.join(detected_attributes)}")

def run_face_detection():
    """Executa detecção facial completa na imagem processada"""
    
    bucket_name = 'staging-face-text'
    image_key = 'processed_Pasted image 20250608090919.png'
    
    print("👥 INICIANDO DETECÇÃO FACIAL")
    print("="*50)
    
    # Executar detecção
    faces = detect_faces_in_image(bucket_name, image_key)
    
    if faces:
        print(f"\n🎉 SUCESSO! {len(faces)} face(s) detectada(s)")
        print("="*50)
        
        # Analisar cada face detectada
        for i, face in enumerate(faces, 1):
            analyze_face_details(face, i)
        
        # Estatísticas gerais
        print(f"\n📊 ESTATÍSTICAS GERAIS:")
        confidences = [face['Confidence'] for face in faces]
        print(f"   🎯 Confiança média: {sum(confidences)/len(confidences):.2f}%")
        print(f"   📈 Confiança máxima: {max(confidences):.2f}%")
        print(f"   📉 Confiança mínima: {min(confidences):.2f}%")
        
        return faces
    else:
        print("❌ Nenhuma face detectada na imagem")
        print("💡 Isso pode acontecer se:")
        print("   • As faces estão muito pequenas")
        print("   • A qualidade da imagem é baixa")
        print("   • As faces estão parcialmente ocultas")
        return []

# Executar detecção facial
print("🚀 Executando detecção facial na sua imagem...")
detected_faces = run_face_detection()

✅ Cliente Rekognition reconfigurado!
🚀 Executando detecção facial na sua imagem...
👥 INICIANDO DETECÇÃO FACIAL
🔍 Analisando faces na imagem: processed_Pasted image 20250608090919.png
🎯 3 face(s) detectada(s) na imagem

🎉 SUCESSO! 3 face(s) detectada(s)

👤 FACE 1:
   🎯 Confiança geral: 100.00%
   📍 Posição: Left=0.498, Top=0.543
   📏 Dimensões: Width=0.077, Height=0.172
   🎂 Idade estimada: 18-22 anos
   👫 Gênero: Female (confiança: 93.2%)
   😊 Emoções detectadas:
      • CALM: 94.6%
      • HAPPY: 1.1%
      • SURPRISED: 0.1%
   📸 Qualidade da imagem:
      • Brilho: 85.7
      • Nitidez: 20.9

👤 FACE 2:
   🎯 Confiança geral: 99.99%
   📍 Posição: Left=0.381, Top=0.583
   📏 Dimensões: Width=0.045, Height=0.112
   🎂 Idade estimada: 19-23 anos
   👫 Gênero: Female (confiança: 99.2%)
   😊 Emoções detectadas:
      • HAPPY: 100.0%
      • SURPRISED: 0.0%
      • CALM: 0.0%
   📸 Qualidade da imagem:
      • Brilho: 90.4
      • Nitidez: 7.6
   👁️  Atributos detectados: Smile (97.9%)

👤 FACE 3